In [1]:
import pandas as pd
import os
from dotenv import load_dotenv
load_dotenv()
import requests
import json

In [19]:
#pip install python-dotenv

In [18]:
sensores_actualizar = pd.read_json('sensores.json')
sensores_actualizar

,sensor_index
0,27552
1,27597


In [4]:
PURPLEAIR_GROUP_ID = os.getenv('PURPLEAIR_GROUP_ID')
PURPLEAIR_READ_KEY = os.getenv('PURPLEAIR_READ_KEY')
PURPLEAIR_WRITE_KEY = os.getenv('PURPLEAIR_WRITE_KEY')

In [22]:
def sensores_actuales():    
    api_url = f'https://api.purpleair.com/v1/groups/{PURPLEAIR_GROUP_ID}'
    headers = {'X-API-Key': PURPLEAIR_READ_KEY}
    response = requests.get(api_url, headers=headers)
    members = response.json()['members']
    sensor_index = [ sub['sensor_index'] for sub in members ]
    member_id = [ sub['id'] for sub in members ]
    return sensor_index, member_id

In [21]:
def adicionar_sensor(sensor_index):    
    api_url = f'https://api.purpleair.com/v1/groups/{PURPLEAIR_GROUP_ID}/members'
    headers = {'Content-Type': 'application/json','X-API-Key': PURPLEAIR_WRITE_KEY}
    data = {'sensor_index': sensor_index}
    response = requests.post(api_url, data=json.dumps(data), headers=headers)
    return response.status_code

In [19]:
def eliminar_sensor(member_id):    
    api_url = f'https://api.purpleair.com/v1/groups/{PURPLEAIR_GROUP_ID}/members/{member_id}'
    headers = {'Content-Type': 'application/json','X-API-Key': PURPLEAIR_WRITE_KEY}
    response = requests.delete(api_url, headers=headers)
    return response.status_code

204

In [20]:
sensor_index, member_id = sensores_actuales()
sensores_act = pd.DataFrame({'sensor_index': sensor_index, 'member_id': member_id})
sensores_act

,sensor_index,member_id
0,27552,163980
1,27597,198797
